In [ ]:
import pandas as pd
from konlpy.tag import Twitter

In [ ]:
# 사진 제외 : 87%
# 사진 포함 : 88%

In [ ]:
df_naver = pd.read_csv('A_V_Adj_C_df_naver_post_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep = pd.read_csv('A_V_Adj_C_df_dbdbdeep_post_table.csv',sep='\t',encoding='utf-8')
df_naver_page = pd.read_csv('A_V_Adj_C_df_naver_page_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep_page = pd.read_csv('A_V_Adj_C_df_dbdbdeep_page_table.csv',sep='\t',encoding='utf-8')

In [ ]:
df_dbdbdeep.groupby(by='Category')['label'].count()

In [ ]:
df_naver.groupby(by='Category')['label'].count()

In [ ]:
df_naver = df_naver.dropna(subset=['Embedding_text'])
df_dbdbdeep = df_dbdbdeep.dropna(subset=['Embedding_text'])

In [ ]:
# 사진 포함 33
df_naver['Embedding_text'] = df_naver['Embedding_text'].apply(lambda x : " ".join(x.split(' ')[:-20]) if len(x.split(' ')) > 33 else x )

In [ ]:
from sklearn.utils import shuffle
df_naver = shuffle(df_naver)

In [ ]:
df_dbdbdeep.shape

In [ ]:
df_naver = df_naver.iloc[:6208]

In [ ]:
Total = pd.concat([df_naver,df_dbdbdeep,df_naver_page,df_dbdbdeep_page])

In [ ]:
Total = Total.dropna(subset=['Embedding_text'])

In [ ]:
Total

## Doc2vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
Total = Total[['Embedding_text','label']]

In [ ]:
text = Total['Embedding_text'].tolist()

In [ ]:
y = Total['label'].tolist()

In [ ]:
sentences = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text)]

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [ ]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=300,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=20,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
)

In [ ]:
doc_vectorizer.build_vocab(sentences)

In [ ]:
print(str(doc_vectorizer))

In [ ]:
doc_vectorizer.corpus_count*0.03

In [ ]:
doc_vectorizer.iter

In [ ]:
import time
start = time.time()
for epoch in range(10):
    doc_vectorizer.train(sentences, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time.time()
print("During Time: {}".format(end-start))

In [ ]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in sentences]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train , y , test_size=0.33, random_state=42)

## SVM

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

# MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=10,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [ ]:
start = time.time()
mlp_clf.fit(X_train, y)
end = time.time()
print('Time: {:f}s'.format(end-start))

In [ ]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))